<a href="https://colab.research.google.com/github/nabilLearns/diambra-arena-bot/blob/master/Diambra_DQN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from matplotlib import pyplot as plt
import torch
import torch.nn as nn
import math
import cv2 

In [ ]:
'''
Q = Our net's output
x = image
a = action
r = reward
s = sequence
phi = processed s
gamma = discount factor
D = replay buffer
'''

In [ ]:
# preprocessing
# need to rescale image, generate

'''


Moves (0-8): (No-Move, Left, Left+Up, Up, Up+Right, Right, Right+Down, Down, Down+Left) 
Attacks (0-3): (No-Attack, Hold, Punch, Kick)


EXAMPLE INPUT:
actions = env.action_space
observation, reward, done, info = env.step(actions)
observation['frame'] = frame array shape = (480, 512, 3)
observation['P1'] = {'oppChar1': 4, 'ownChar1': 8, 'oppChar': 4, 'ownChar': 8, 'oppHealth': 143, 'ownHealth': 187, 'oppSide': 0, 'ownSide': 1, 'oppWins': 1, 'ownWins': 0, 'actions': {'move': 5, 'attack': 7}}
observation['stage'] = 1


info = {'roundDone': False, 'stageDone': False, 'gameDone': False, 'epDone': False}
Info: {'roundDone': False, 'stageDone': False, 'gameDone': False, 'epDone': False}
observation["frame"].shape: (480, 512, 3)
observation["stage"]: 1
observation["P1"]["ownChar1"]: Kasumi
observation["P1"]["oppChar1"]: Bass
observation["P1"]["ownChar"]: Kasumi
observation["P1"]["oppChar"]: Bass
observation["P1"]["ownHealth"]: 84
observation["P1"]["oppHealth"]: 23
observation["P1"]["ownSide"]: 1
observation["P1"]["oppSide"]: 1
observation["P1"]["ownWins"]: 0
observation["P1"]["oppWins"]: 0
observation["P1"]["actions"]: {'move': 6, 'attack': 0}
Reward: 0.9423076923076923

Done: False
'''


#In: 6 of (480, 512, 3)
#To net: (6, 1, 256, 256)
#Out of imagenet: (6, hidden_size)
#Out of timenet: (hidden_size2)
#Out of metanet: (hidden_size3)
#Out of final net: one-val
#One-val//8 = move, One-vack

import cv2
# GREYSCALE
img = cv2.imread('sample_out_2.png')
small = cv2.resize(img, (256,256)) 
img_gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
print(img_gray.shape)
plt.imshow(img_gray)
plt.show()

In [ ]:
# Image - Network
class ImageNetwork(nn.Module):
    """
    SAMPLE, NEED TO CHANGE
    """
    def output_shape(size, kernel_size=5, stride=2):
        return math.floor(((size - (kernel_size - 1) - 1)/stride) + 1)

    def __init__(self, input_shape, hidden_size1):
        super(ImageNetwork, self).__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(input_shape[0], 32, kernel_size=8, stride=4),
            nn.ReLU(),
            nn.Conv2d(32, 64, kernel_size=4, stride=2),
            nn.ReLU(),
            nn.Conv2d(64, 64, kernel_size=3, stride=1),
            nn.ReLU(),
            nn.Flatten(start_dim=1)
        )

        convw, convh = input_shape[0], input_shape[1]
        kernel_sizes = [8, 4, 3]
        stride_vals = [4, 2, 1]
        num_layers = 3
        for i in range(num_layers):
            convw = self.output_shape(convw, kernel_sizes[i], stride_vals[i])
            convh = self.output_shape(convh, kernel_sizes[i], stride_vals[i])
        convc = 64
        dim_size = convc*convh*convw
        
        '''
        (N, c_in, h_in, w_in)
        (N, convc, convh, convw)
        '''

        '''
        (N, X)
        (N, hidden_size1)
        '''

        self.fc = nn.Sequential(
            nn.Linear(dim_size, 512),
            nn.ReLU(),
            nn.Linear(512, hidden_size1)
        )

    def forward(self, in_vals):
        out = self.conv(in_vals)
        out = self.fc(out)
        return out

In [ ]:
class TimeLSTM(nn.Module):
    def __init__(self, input_shape, output_shape, hidden_size, num_layers):
        super(TimeLSTM, self).__init__()
        self.size_in = input_shape
        self.size_out = output_shape
        self.num_layers = num_layers
        self.hidden_size = hidden_size

        self.LSTM = nn.LSTM(input_size=self.size_in, hidden_size=self.hidden_size, num_layers=self.num_layers)
        self.ac1 = nn.LeakyReLU()
        self.FC = nn.Linear(in_features = self.hidden_size, out_features = self.size_out)
        self.ac2 = nn.ReLU()

    def forward(self, in_vals):
        out, (h_n, c_n) = self.LSTM(in_vals)
        output = h_n[-1,:]
        output = self.ac1(output)
        output = self.FC(output)
        output = self.ac2(output)
        return output

In [ ]:
class metaData(nn.Module):
    def __init__(self, input_shape, hidden_sz, n_outputs):
        """
        input_shape = 12
        """
        super(metaData, self).__init__()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(input_shape[0], hidden_sz),
            nn.ReLU(),
            nn.Linear(hidden_sz, hidden_sz) ,
            nn.ReLU(),
            nn.Linear(hidden_sz, n_outputs)
        )
     
    def forward(self, input):
        return self.linear_relu_stack(input)  
        
        

#observation['P1'] = {'oppChar1': 4, 'ownChar1': 8, 'oppChar': 4, 'ownChar': 8, 'oppHealth': 143, 'ownHealth': 187, 'oppSide': 0, 'ownSide': 1, 'oppWins': 1, 'ownWins': 0, 'actions': {'move': 5, 'attack': 7}}
#observation['stage'] = 1
        

In [ ]:
class finalNetwork(nn.Module):
    def __init__(self, input_shape, hidden_sz, n_actions=72):
        super(finalNetwork, self).__init__()
        self.dense_network = nn.Sequential(
            nn.Linear(input_shape[0], hidden_sz),
            nn.ReLU(),
            nn.Linear(hidden_sz, n_actions)
        )
            
    def forward(self, input):
        return self.dense_network(input)
        
    def predict(self, input):  
        return self.forward(input).argmax() 

In [ ]:
def preprocess(observation, past_5_frames):
    """given an observation, send it through the model pipeline and give us the best action"""
    # PROCESS(observation["frame"])
        
    current_frame = observation['frame']
    metaData = observation['P1']

    def get_input_to_metanet(observation):
        lambda action_to_num: action['move']*8 + action['attack']
        metaData = [value for key, value in observation['P1'].items() if not key == 'actions']
        metaData += action_to_num(observation['P1']['actions'])  
        metaData += observation['stage']  

In [ ]:
# DQN Algorithm

REPLAY_MEMORY = []

def SELECT_ACTION(epsilon):
    """Selects random action w/ prob epsilon, otherwise picks the action of highest value from the Q-Network"""
    
    action_type = np.random.choice([0, 1], p=[epsilon, 1-epsilon])
    if action_type == 0:
        action = env.action_space.sample()
    else:
        input = preprocess(observation)
        action = model.predict(input)f (observation)   # select best action from output of Q net-work

def LOSS(y, Q):
    return nn.functional.mse_loss(y, Q)

def RUN_DQN_ALGORITHM(num_episodes, num_time_steps, eps=1, min_eps=0.05):
    """
    Runs through DQN algo. as described in https://www.cs.toronto.edu/~vmnih/docs/dqn.pdf

    Parameters
    ----------
    eps (float): probability of selecting a random action. this can decay as the learning process progresses (i.e. do more exploration initially, then less later on) 
    """

    for episode in range(num_episodes):
        observation = env.reset()
        for step in range(num_time_steps):
            eps = max(eps*0.99, min_eps)
            action = SELECT_ACTION(eps)
            observation, reward, done, info = env.step(action)

            # store transition in REPLAY_MEMORY
            # sample random minibatch of transitions from REPLAY_MEMORY
            # set target value y_i according to eqn in paper
            # perform grad. desc. on LOSS(y_i, Q_value)



transition = [state, action, reward, next_state]
REPLAY_MEMORY.append(transition)